In [28]:
# Gerekli kütüphaneleri yükle
import os
import pandas as pd
import torch
from PIL import Image
from torch.utils.data import Dataset
from transformers import (
    VisionEncoderDecoderModel,
    AutoTokenizer,
    AutoFeatureExtractor,
    Seq2SeqTrainer,
    Seq2SeqTrainingArguments
)

# GPU kontrolü
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Kullanılan cihaz: {device}")

# Verileri yükle
train_df = pd.read_csv("/kaggle/input/obbsdata/train.csv")
image_folder = "/kaggle/input/obbsdata/train/train/"

# DataFrame indeksini sıfırla
train_df = train_df.reset_index(drop=True)

# Eğitim için veri kümesi sınıfı
class ImageCaptionDataset(Dataset):
    def __init__(self, df, image_folder, feature_extractor, tokenizer, max_length=128):
        self.df = df
        self.image_folder = image_folder
        self.feature_extractor = feature_extractor
        self.tokenizer = tokenizer
        self.max_length = max_length
        
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, idx):
        # DataFrame'den doğru şekilde veri çek
        row = self.df.iloc[idx]
        img_path = os.path.join(self.image_folder, str(row['image_id']) + ".jpg")
        caption = str(row['caption'])
        
        try:
            image = Image.open(img_path).convert("RGB")
            pixel_values = self.feature_extractor(image, return_tensors="pt").pixel_values[0]
            
            labels = self.tokenizer(
                caption, 
                max_length=self.max_length, 
                padding="max_length", 
                truncation=True, 
                return_tensors="pt"
            ).input_ids[0]
            
            return {"pixel_values": pixel_values, "labels": labels}
        except Exception as e:
            print(f"Error loading image {img_path}: {str(e)}")
            # Return a dummy sample if there's an error
            dummy_image = Image.new("RGB", (224, 224))
            pixel_values = self.feature_extractor(dummy_image, return_tensors="pt").pixel_values[0]
            labels = self.tokenizer(
                "dummy caption", 
                max_length=self.max_length, 
                padding="max_length", 
                truncation=True, 
                return_tensors="pt"
            ).input_ids[0]
            return {"pixel_values": pixel_values, "labels": labels}

# Model ve tokenizer'ı yükle
model_name = "nlpconnect/vit-gpt2-image-captioning"
feature_extractor = AutoFeatureExtractor.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = VisionEncoderDecoderModel.from_pretrained(model_name).to(device)

# Veri kümesini hazırla
dataset = ImageCaptionDataset(train_df, image_folder, feature_extractor, tokenizer)

# Eğitim ve validasyon için ayır (80-20)
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = torch.utils.data.random_split(
    dataset, [train_size, val_size], 
    generator=torch.Generator().manual_seed(42)
)

# Eğitim ayarları
training_args = Seq2SeqTrainingArguments(
    output_dir="./output",
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    eval_steps=500,
    save_steps=500,
    logging_steps=100,
    learning_rate=5e-5,
    weight_decay=0.01,
    fp16=True if device == "cuda" else False,
    logging_dir="./logs",
    report_to="none",
    save_total_limit=2,
)

# Trainer oluştur
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=feature_extractor,
)

# Eğitimi başlat
print("Eğitim başlıyor...")
trainer.train()
print("Eğitim tamamlandı!")

# Modeli kaydet
model.save_pretrained("./fine_tuned_model")
tokenizer.save_pretrained("./fine_tuned_model")
print("Model kaydedildi.")

Kullanılan cihaz: cuda


/usr/local/lib/python3.11/dist-packages/transformers/models/vit/feature_extraction_vit.py:28: FutureWarning: The class ViTFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use ViTImageProcessor instead.
  warnings.warn(
Config of the encoder: <class 'transformers.models.vit.modeling_vit.ViTModel'> is overwritten by shared encoder config: ViTConfig {
  "architectures": [
    "ViTModel"
  ],
  "attention_probs_dropout_prob": 0.0,
  "encoder_stride": 16,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.0,
  "hidden_size": 768,
  "image_size": 224,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "model_type": "vit",
  "num_attention_heads": 12,
  "num_channels": 3,
  "num_hidden_layers": 12,
  "patch_size": 16,
  "pooler_act": "tanh",
  "pooler_output_size": 768,
  "qkv_bias": true,
  "torch_dtype": "float32",
  "transformers_version": "4.51.3"
}

Config of the decoder: <class 'transformers.models.gpt2.modeling_

Eğitim başlıyor...


Step,Training Loss
100,0.533200
200,0.439700
300,0.426000
400,0.421600
500,0.403200
600,0.403900
700,0.395400
800,0.392000
900,0.395400
1000,0.383200


Eğitim tamamlandı!
Model kaydedildi.


In [29]:
import os
import pandas as pd
import torch
from PIL import Image
from transformers import (
    VisionEncoderDecoderModel,
    AutoTokenizer,
    ViTFeatureExtractor,
    GPT2Tokenizer
)

# GPU kontrolü
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Kullanılan cihaz: {device}")

# Model ve bileşenleri yükleme
model_path = "./fine_tuned_model"

# Orijinal modelin bileşenlerini yükle
encoder_model = "google/vit-base-patch16-224-in21k"
decoder_model = "gpt2"

feature_extractor = ViTFeatureExtractor.from_pretrained(encoder_model)
tokenizer = GPT2Tokenizer.from_pretrained(decoder_model)

# Eğitilmiş modeli yükle
model = VisionEncoderDecoderModel.from_pretrained(model_path).to(device)

# Tokenizer özel ayarları
tokenizer.pad_token = tokenizer.eos_token

# Test verilerini yükle
test_df = pd.read_csv("/kaggle/input/obbsdata/test.csv")
image_folder = "/kaggle/input/obbsdata/test/test/"

# Resimler için caption üretme fonksiyonu (geliştirilmiş versiyon)
def generate_caption(image_path):
    try:
        # Resmi yükle ve işle
        image = Image.open(image_path).convert("RGB")
        
        # Özellik çıkarımı
        pixel_values = feature_extractor(
            images=image, 
            return_tensors="pt"
        ).pixel_values.to(device)
        
        # Caption üretme
        output_ids = model.generate(
            pixel_values,
            max_length=128,
            num_beams=4,
            temperature=0.9,
            early_stopping=True,
            no_repeat_ngram_size=2
        )
        
        # Caption'ı decode et
        caption = tokenizer.decode(
            output_ids[0], 
            skip_special_tokens=True,
            clean_up_tokenization_spaces=True
        )
        
        return caption.capitalize()  # İlk harfi büyük yap
    
    except Exception as e:
        print(f"Hata oluştu ({image_path}): {str(e)}")
        return "A photo"  # Varsayılan caption

# Test seti için tahminler oluştur
predictions = []


for idx, row in test_df.iterrows():
    image_id = row['image_id']
    img_path = os.path.join(image_folder, str(image_id) + ".jpg")  # self yerine image_folder

    caption = generate_caption(img_path)  # image_path yerine img_path olmalı
    
    predictions.append({"image_id": image_id, "caption": caption})
    
    if (idx + 1) % 100 == 0:
        print(f"{idx + 1}/{len(test_df)} resim işlendi")


# Sonuçları CSV'ye kaydet
submission_df = pd.DataFrame(predictions)
submission_df.to_csv("submission.csv", index=False)

print("\nSubmission dosyası başarıyla oluşturuldu!")
print("İlk 5 tahmin:")
print(submission_df.head())

Kullanılan cihaz: cuda


/usr/local/lib/python3.11/dist-packages/transformers/models/vit/feature_extraction_vit.py:28: FutureWarning: The class ViTFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use ViTImageProcessor instead.
  warnings.warn(
Config of the encoder: <class 'transformers.models.vit.modeling_vit.ViTModel'> is overwritten by shared encoder config: ViTConfig {
  "architectures": [
    "ViTModel"
  ],
  "attention_probs_dropout_prob": 0.0,
  "encoder_stride": 16,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.0,
  "hidden_size": 768,
  "image_size": 224,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "model_type": "vit",
  "num_attention_heads": 12,
  "num_channels": 3,
  "num_hidden_layers": 12,
  "patch_size": 16,
  "pooler_act": "tanh",
  "pooler_output_size": 768,
  "qkv_bias": true,
  "torch_dtype": "float32",
  "transformers_version": "4.51.3"
}

Config of the decoder: <class 'transformers.models.gpt2.modeling_

100/3771 resim işlendi
200/3771 resim işlendi
300/3771 resim işlendi
400/3771 resim işlendi
500/3771 resim işlendi
600/3771 resim işlendi
700/3771 resim işlendi
800/3771 resim işlendi
900/3771 resim işlendi
1000/3771 resim işlendi
1100/3771 resim işlendi
1200/3771 resim işlendi
1300/3771 resim işlendi
1400/3771 resim işlendi
1500/3771 resim işlendi
1600/3771 resim işlendi
1700/3771 resim işlendi
1800/3771 resim işlendi
1900/3771 resim işlendi
2000/3771 resim işlendi
2100/3771 resim işlendi
2200/3771 resim işlendi
2300/3771 resim işlendi
2400/3771 resim işlendi
2500/3771 resim işlendi
2600/3771 resim işlendi
2700/3771 resim işlendi
2800/3771 resim işlendi
2900/3771 resim işlendi
3000/3771 resim işlendi
3100/3771 resim işlendi
3200/3771 resim işlendi
3300/3771 resim işlendi
3400/3771 resim işlendi
3500/3771 resim işlendi
3600/3771 resim işlendi
3700/3771 resim işlendi

Submission dosyası başarıyla oluşturuldu!
İlk 5 tahmin:
   image_id                                            caption
0